<center>
<img src="https://raw.githubusercontent.com/MLGlobalHealth/StatML4PopHealth/main/practicals/resources/logos/imperial.png" width="250" vspace="8"/>
<img src="https://raw.githubusercontent.com/MLGlobalHealth/StatML4PopHealth/main/practicals/resources/logos/mlgh.png" width="220" hspace="50" vspace="5"/>
<img src="https://raw.githubusercontent.com/MLGlobalHealth/StatML4PopHealth/main/practicals/resources/logos/ammi.png" width="190"/>

<font size="6">Modern Statistics and Machine Learning
for Population Health in Africa </font>

<font size="4">24th - 28th March 2025</font>

</center>



# The City of Cape Town Air Quality Monitoring (AQM) Network

### Tristan Naidoo and Sahoko Ishida

The AQM network consists of multiple stations equipped with sensors to measure both atmospheric pollutants and environmental parameters. Examples of atmospheric pollutants include particulate matter (e.g., PM2.5 and PM10) and sulfur dioxide (SO₂), while environmental parameters include humidity, wind speed, and wind direction. See this [review](https://www.sciencedirect.com/science/article/pii/S0301479721005727) by Singh et al. for more details on AQM networks.

The Western Cape AQM network extends from St. Helena Bay to George. The first station, Molteno, was commissioned in 1992, and the network has since expanded to a total of 14 monitoring stations.

In this notebook, we will focus on a subset of the City of Cape Town AQM stations, which form part of the broader network. An example of the network is shown below, with the stations of interest highlighted in green.
<br>
<center>
<img src="https://raw.githubusercontent.com/MLGlobalHealth/StatML4PopHealth/refs/heads/main/practicals/day1/practical1/images/AQM_network.png" width=353 height=500/>
</center>

([source](https://saaqis.environment.gov.za/Lekgotla%20Proceedings/2019/2019_2.2-overview-of-aqm_in-the-wc.pdf) - the original image was edited to highlight the stations we have data for)

Our focus will be on wind speed, which plays a key role in the dispersion of atmospheric pollutants. Higher wind speeds are typically associated with lower pollutant concentrations, as they help disperse airborne contaminants. See for example, this [study](https://www.nature.com/articles/s41598-020-65391-5)  by Yang et al. that investigates factors influencing PM mass concentration in Shenyang, China and finds that wind speed was the most significant meteorological factor.

# Processing the raw data

This notebook will aim to prepare hourly wind speed data to be modelled by a basic linear regression which you will cover in the next practical. In addition, it will help you dust off some of your python data wrangling skills which you'll need to process data given its tricky format!

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from pathlib import Path

sns.set_theme(style="whitegrid")

We begin by reading the data, which was downloaded from the City of Cape Town Open Data portal (see [here](https://odp-cctegis.opendata.arcgis.com/documents/31ef242a23484e79bbb19d6b29203179/about) for more). Each year in kept in an individual .xlsx file. Each file has wind speed measured in metres per second (m/s) and wind direction measured in degrees (deg) across different AQM stations. In total we have data for 7 AQM stations.

If you look at the raw data you will that the data format changes in 2017, so we read the individual csvs into two seperate lists, which we then combine.

In [ ]:
# Get input data
import subprocess

resource_urls= [f"https://github.com/MLGlobalHealth/StatML4PopHealth/raw/refs/heads/main/practicals/day1/practical1/data/raw/Wind_direction_and_speed_{year}.xlsx"
 for year in range(2014, 2021)]

# Note this will overwrite existing files, remove -O if that is not desired
# -L tells curl to follow redirrections
for url in resource_urls:
   subprocess.run(["curl", "-LO", url], check=True)

In [ ]:
df_2014_2016_list = [pd.read_excel(f"Wind_direction_and_speed_{year}.xlsx", skiprows=2, header=[0, 1, 2]) for year in range(2014, 2017)]
df_2017_2020_list = [pd.read_excel(f"Wind_direction_and_speed_{year}.xlsx", skiprows=2, header=[0, 1, 2])[:-8] for year in range(2017, 2021)]

df_list = df_2014_2016_list + df_2017_2020_list

Let's see what a single DataFrame looks like.

In [ ]:
example_df = df_list[0]
example_df.head()

As mentioned, the data has a tricky format. We'll first try to process a single DataFrame before we tackle our list.

The DataFrame has a three-level column index. Let's clean this up a bit

In [ ]:
# Clears the Unnamed column names
example_df = example_df.rename(columns={example_df.columns[0][0]: "timestamp",
                                        example_df.columns[0][1]:"",
                                        example_df.columns[0][2]:""})

# Labels each level of the index
example_df.columns.names = ["location", "parameter", "unit"]

example_df.head()

Now, let's convert this data into long format.

In [ ]:
# Convert the data to a long format and keep the first column as the ID variable
example_df_long = example_df.melt(id_vars=[("timestamp", "", "")], value_name="value")
example_df_long = example_df_long.rename(columns={example_df_long.columns[0]: "timestamp"})

example_df_long.head()

Finally, let's pivot the `parameter` column into two separate columns.

In [ ]:
example_df_wide = example_df_long.pivot(index=["timestamp", "location"], columns="parameter", values="value").reset_index()
example_df_wide.head()

The data looks much better! Instead of repeating the steps for each DataFrame in the list, let's create a function which combines the previous steps.

In [ ]:
def process_df(df):
    df = df.rename(columns={df.columns[0][0]: "timestamp", df.columns[0][1]:"", df.columns[0][2]:""})
    df.columns.names = ["location", "parameter", "unit"]

    df_long = df.melt(id_vars=[("timestamp", "", "")], value_name="value")
    df_long = df_long.rename(columns={df_long.columns[0]: "timestamp"})

    df_wide = df_long.pivot(index=["timestamp", "location"], columns="parameter", values="value").reset_index()

    return df_wide

We can use this helper function to efficiently process each DataFrame in our list and then we can combine the result into a single DataFrame.

In [ ]:
processed_df_list = [process_df(df) for df in df_list]

In [ ]:
complete_wind_df = pd.concat(processed_df_list)

In [ ]:
# The shape of the processed data
complete_wind_df.shape

In [ ]:
complete_wind_df.head()

# Cleaning the processed data

## Wind speed and wind direction

Looking at the wind speed and wind direction, you’ll notice that missing values have been coded as the string 'NoData'. This will be difficult to work with. Let’s check if there are any other strings and see how many there are in each of these columns.

In [ ]:
# Rename the columns so that that they're each to work with
complete_wind_df.rename(columns={"Wind Dir V": "wind_dir", "Wind Speed V": "wind_speed"}, inplace=True)

In [ ]:
# Checks if the value in each column is a string
is_string_filter_df = complete_wind_df[["wind_dir", "wind_speed"]].map(lambda x: isinstance(x, str))

In [ ]:
# String value data frames for each variable
wind_dir_string_df = complete_wind_df["wind_dir"][is_string_filter_df["wind_dir"]]
wind_speed_string_df = complete_wind_df["wind_speed"][is_string_filter_df["wind_speed"]]

In [ ]:
# Missing wind direction value counts
wind_dir_string_df.value_counts()

In [ ]:
# Missing wind speed value counts
wind_speed_string_df.value_counts()

In [ ]:
total_missing = sum(is_string_filter_df["wind_dir"] | is_string_filter_df["wind_speed"])

print(f"In total there are {len(wind_speed_string_df)} missing wind speed values "
      f"and {len(wind_dir_string_df)} missing wind direction values."
      f"\nThe combined number of missing values is {total_missing}")

To work with these columns it would be better if they were numeric. To do this lets replace the string values with `NA` values.

In [ ]:
unique_strings = set()
unique_strings.update(wind_dir_string_df.unique())
unique_strings.update(wind_speed_string_df.unique())

In [ ]:
unique_strings

In [ ]:
# np.nan keeps column as numeric when plotting (confirm)
complete_wind_df = complete_wind_df.infer_objects(copy=False).replace(unique_strings, np.nan)

In [ ]:
# The number of na values should correspond to our earlier string counts
complete_wind_df.isna().apply(sum)

In [ ]:
# As an additional check, lets confirm that if a value isn't a float then it's na
complete_wind_df[["wind_dir", "wind_speed"]].map(lambda x: isinstance(x, (int, float)) or  pd.isna(x)).all()

## Location

Since we're loading the data over multiple years, it's possible that some of the station names may have updated.

In [ ]:
complete_wind_df.location.unique()

Indeed, it looks like this is the case for Somerset West, lets update the strings to match.

In [ ]:
complete_wind_df.loc[complete_wind_df.location=="Somerset-West AQM Site", "location"]  = "Somerset West AQM Site"

In [ ]:
complete_wind_df.location.unique()

Much better! Now we have a unique set of locations.

## Creating datetime features

Now that the data we have is cleaned, we can create some time-related features.

In [ ]:
# Covert the timestamp to a datetime object to make creating features easier
complete_wind_df["timestamp"] = pd.to_datetime(complete_wind_df["timestamp"], format="%d/%m/%Y %H:%M")

In [ ]:
complete_wind_df = complete_wind_df.sort_values("timestamp")

In [ ]:
complete_wind_df['date'] = complete_wind_df['timestamp'].dt.date

complete_wind_df['hour'] = complete_wind_df['timestamp'].dt.hour
complete_wind_df['dow'] = complete_wind_df['timestamp'].dt.strftime('%w')
complete_wind_df['dow_hour'] = complete_wind_df['timestamp'].dt.strftime('%w-%H')

complete_wind_df['month'] = complete_wind_df['timestamp'].dt.month
complete_wind_df['year'] = complete_wind_df['timestamp'].dt.year
complete_wind_df['month_year'] = complete_wind_df['timestamp'].dt.strftime('%Y-%m')

Lets save full dataset with cleaned features. To do this, lets first mount our google drive and specify the output folder.

In [ ]:
# specify ouput location
from google.colab import drive
drive.mount('/content/drive')

# Adjust this as required - this is where your output will be stored.
output_dir = Path(*["drive", "MyDrive", "StatML4PopHealth", "practical1"])
output_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
complete_wind_df.to_csv(output_dir.joinpath(
    "cleaned_wind_data_date_features.csv"), index=False)

If you would like to read in the data you need to recovert the timestamp to a datetime object.

In [ ]:
# For reading in the data
complete_wind_df = pd.read_csv(output_dir.joinpath(
    "cleaned_wind_data_date_features.csv"))
complete_wind_df.timestamp =  pd.to_datetime(complete_wind_df["timestamp"], format="%Y-%m-%d %H:%M:%S")

# Plotting the data

To assist with looking at the data, we've created a simple helper function above that makes plotting the time variables easier.

Quick plot options: `date`, `hour`, `dow`, `dow_hour`, `month`, `month_year`, `year`.

If you have time, unrelated to selecting the year to model, feel free to explore the different options.
<br>Can you see any patterns in wind speed when the plot options are `hour` and `month`?

In [ ]:
def quick_plot(df, x, y, group_var="location", ncols=2, col_width=5):
    grouped = df.groupby(group_var)
    nrows = math.ceil(len(grouped)/ncols)
    n_unreq_axes = (nrows*ncols) - len(grouped)

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(col_width*ncols, nrows*4))
    fig.suptitle(f"Wind speed (m/s) at {group_var}:", fontsize=15)
    for (group_name, group), ax in zip(grouped, axes.flatten()):
        group.plot(x=x,y=y,ax=ax)
        ax.set_title(f"{group_name}\n n={group.shape[0]}")
        ax.legend().set_visible(False)

    for i in range(1, n_unreq_axes+1):
        axes.flatten()[-i].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
group_var = "location"
grouped_df_dict = {"date": complete_wind_df.groupby(["date", group_var])["wind_speed"].mean().reset_index(),
                   "hour": complete_wind_df.groupby(["hour", group_var])["wind_speed"].mean().reset_index(),
                   "dow_hour": complete_wind_df.groupby(["dow_hour", group_var])["wind_speed"].mean().reset_index(),
                   "month": complete_wind_df.groupby(["month", group_var])["wind_speed"].mean().reset_index(),
                   "year": complete_wind_df.groupby(["year", group_var])["wind_speed"].mean().reset_index(),
                   "month_year": complete_wind_df.groupby(["month_year", group_var])["wind_speed"].mean().reset_index()}

In [ ]:
quick_plot_var = "year"

In [ ]:
quick_plot(grouped_df_dict[quick_plot_var], quick_plot_var, "wind_speed" , group_var, ncols=4, col_width=4)

# Extra: creating a modelling dataset for a single year

In the modelling notebook we will look at the hourly windspeed recordings at a selected site ('Bothasig AQM Site'), between 1st January, 2020 and 21st January 2020. However which year should you select if you wanted to model the daily wind speed for a single year?

Unfortunately, our dataset contains a lot of missing values, which you may have noticed when we replaced strings with `NA` values. Additionally, Khayelitsha, Somerset West, and Bellville only have data available from 2016 onwards. As a result, we'll focus on a single year and exclude 2014 and 2015 as candidates for our modeling.

In [ ]:
complete_wind_filtered_df = complete_wind_df[~(complete_wind_df["year"].isin([2014, 2015]))]

In [ ]:
missingness_year_df = (complete_wind_filtered_df.
                       groupby(["year"])["wind_speed"].
                       apply(lambda x: round(x.isna().sum()/x.shape[0],2)).reset_index())

In [ ]:
missingness_year_df.rename({"wind_speed": "pers_missing"}, axis=1)

2017, 2019, and 2020 look like reasonable candidates. Let us take a closer look at each year.

In [ ]:
complete_wind_filtered_df = complete_wind_filtered_df[(complete_wind_filtered_df["year"].isin([2017, 2019, 2020]))]

In [ ]:
missingness_year_location_df = complete_wind_filtered_df[["year", "location", "timestamp","wind_speed"]].groupby(["year", "location"]).count()
missingness_year_location_df["pers_missing"] = round(1-missingness_year_location_df["wind_speed"]/missingness_year_location_df["timestamp"],2)

In [ ]:
missingness_year_location_df[["pers_missing"]]

Looking at the table above, only two locations in 2019 and 2020 have more than 35% of their values missing. Let's plot the data to see which year looks better across the different locations.

Based on the plots which plots which year do you think is best and why? Scroll down for our answer!
<br> 
<br> 
<br> 
<br> 
<br> 
<br> 
<br> 
<br> 
<br> 
<br> 
<br> 
<br> 
<br> 
<br> 
<br> 
<br> 
<br> 
<br> 
<br>
<br>
<br>
<br>
<br>
<br>
Looking at the yearly plot, we notice a data error for Goodwood in 2020. Based on this, we have chosen 2019 as the year to model. However, feel free to challenge this if you think 2020 makes more sense - come chat with us and share your reasoning! :)"